In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("test_app").getOrCreate()

Looking at the data present and removing the timestamp

In [2]:
data_for_model=spark.read.csv("./data/ratings.csv", header=True, inferSchema=True)

In [3]:
data_for_model=data_for_model.drop("timestamp")

In [4]:
ratings=data_for_model
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.30% empty.


In [5]:
from pyspark.sql.functions import count, avg, min
# Min num ratings for movies
print("Movie with the fewest ratings: ")
ratings.groupBy("movieId").count().select(min("count")).show()

# Avg num ratings per movie
print("Avg num ratings per movie: ")
ratings.groupBy("movieId").count().select(avg("count")).show()

# Min num ratings for user
print("User with the fewest ratings: ")
ratings.groupBy("userId").count().select(min("count")).show()

# Avg num ratings per users
print("Avg num ratings per user: ")
ratings.groupBy("userId").count().select(avg("count")).show()

Movie with the fewest ratings: 
+----------+
|min(count)|
+----------+
|         1|
+----------+

Avg num ratings per movie: 
+------------------+
|        avg(count)|
+------------------+
|10.369806663924312|
+------------------+

User with the fewest ratings: 
+----------+
|min(count)|
+----------+
|        20|
+----------+

Avg num ratings per user: 
+------------------+
|        avg(count)|
+------------------+
|165.30491803278687|
+------------------+



In [6]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



Model Creation and Evaluation

In [7]:
train_df, test_df = data_for_model.randomSplit(weights=[0.7,0.3], seed=100)


In [8]:
# Configure the ALS model
import setuptools.dist
from pyspark.ml.recommendation import ALS


In [9]:

als = ALS(userCol='userId', itemCol='movieId', ratingCol='rating',
          coldStartStrategy='drop', nonnegative=True,rank=4,seed=0)

model = als.fit(train_df)

In [10]:
predictions = model.transform(test_df)

from pyspark.ml.evaluation import RegressionEvaluator
# Tell Spark how to evaluate predictions
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
# Obtain and print RMSE
rmse = evaluator.evaluate(predictions)
print ("RMSE: ", rmse)


RMSE:  0.8879303777841852


Recommendation for a single user


In [11]:
single_user = test_df.filter(test_df['userId']==1)

In [12]:
reccomendations = model.transform(single_user)

In [13]:
movies=spark.read.csv("./data/movies.csv", header=True, inferSchema=True)


In [14]:
recs_final=reccomendations.orderBy('prediction',ascending=False).limit(5)
recs_final.join(movies,recs_final.movieId==movies.movieId).select(['title','genres']).show(truncate=False)

+-----------------------------------------+---------------------------------------+
|title                                    |genres                                 |
+-----------------------------------------+---------------------------------------+
|Star Wars: Episode IV - A New Hope (1977)|Action|Adventure|Sci-Fi                |
|Silence of the Lambs, The (1991)         |Crime|Horror|Thriller                  |
|Mr. Smith Goes to Washington (1939)      |Drama                                  |
|Princess Bride, The (1987)               |Action|Adventure|Comedy|Fantasy|Romance|
|Office Space (1999)                      |Comedy|Crime                           |
+-----------------------------------------+---------------------------------------+



Recommendations for all users


In [15]:
user_recs=model.recommendForAllUsers(3)
data=user_recs.collect()

In [16]:
for user in data:
    print(user['userId'])
    for rec in user['recommendations']:
        print(rec['movieId'])

1
104875
68945
3379
2
40491
68945
3379
3
3223
25825
25850
4
28
446
3567
5
187717
162414
160644
6
86347
92643
104875
7
89904
96004
3030
8
68945
3379
33649
9
3235
40491
104875
10
86320
68073
112804
11
60943
59018
68945
12
68073
3200
26171
13
68945
3379
60943
14
68945
3379
60943
15
26171
4082
68073
16
68945
3379
40491
17
68945
3379
104875
18
68945
3379
33649
19
68945
3379
40491
20
3235
40491
104875
21
185029
2789
106642
22
86644
446
5222
23
68945
3379
3235
24
68945
3379
60943
25
60943
59018
68945
26
68945
3379
104875
27
68073
3200
142422
28
60943
59018
68945
29
68945
3379
60943
30
60943
59018
68945
31
68073
3200
142422
32
8235
4495
68945
33
96004
89904
68945
34
183897
68945
3379
35
4518
28
8235
36
60943
59018
68945
37
60943
59018
3200
38
104339
86347
123
39
4518
8235
4495
40
68945
3379
33649
41
28
3567
446
42
86347
104875
92643
43
86347
158882
149566
44
40491
8477
148881
45
60943
59018
68945
46
104875
86347
92643
47
26171
4082
68073
48
60943
59018
68945
49
68945
3379
33649
50
89904
96004


In [17]:
from pyspark.sql import Row

to_predict = {
                'userId':[0,0],
                'movieId':[ 954, 457],
                'rating': [3.0, 5.0]} 


new_user_id=0
new_user_ratings = [
    Row(userId=new_user_id, movieId=954, rating=4.0),
    Row(user=new_user_id, movieId=457, rating=5.0),
    Row(user=new_user_id, movieId=33649, rating=3.0),
]

# Create a DataFrame for the new user's ratings
new_user_ratings_df = spark.createDataFrame(new_user_ratings)
        
# Create DataFrame with schema 
df = spark.createDataFrame(new_user_ratings) 

